### Fera formidável 4.1 - " Quem classifica a classe classificadora?"

Alunos: José Victor da Silva Izidório e Lucas Nascimento da Silva

#### Enunciado

Objetivo: altere a modelo neural feita em Python puro para resolver um problema de
classificação. Treine uma modelo neural em um dataset simples de classificação para mostrar
que funciona.

Comentário: aqui é necessário se informar sobre as diferenças de uma modelo neural
classificadora com relação a uma modelo neural regressora. A função de perda, por exemplo,
não poderá ser mais a função de perda dos resíduos quadrados.

### Introdução

Para adaptar a modelo neural para um problema de classificação o ponto de partida inicial é adaptar a classe Valor para fazer duas novas funções: 

* Adicionar a função de ativação ReLU e adicionar uam forma de ativar quando queremos usar a função ReLU. A função de ativação ReLU é definida como: 

$$
\mathrm{ReLU}(x) = 
\begin{cases}
0, & \text{se } x < 0 \\
x, & \text{se } x \geq 0
\end{cases}
$$

Existe algumas vantagens em utilizar a função ReLU no lugar da sigmoid, como a maior simplicidade para calcular e evitar que os valores vão para os extremos ficando muito próximos a zero e dificultando o cálculo dos gradientes. 
 
* Calcular logaritmo para fazer o cálculo da função de perda CrossEntropy. 

Além de adaptar a classe Valor, foi necessário definir uma função para calcular a função de perda CrossEntropy. Definida como:

$$
\mathcal{L}_{\text{binária}}(y, \hat{y}) = - \left[ y \log(\hat{y}) + (1 - y) \log(1 - \hat{y}) \right]
$$

Onde:

$y$ = Valor real 

$\hat{y}$ = Valor previsto

A segunda adaptação que fizemos foi apenas aplicar o OneHotEncoder nos dados de classificação para colocá-los na modelo neural.

### Importando as bibliotecas

In [64]:
import random
import math
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from Fera_4_1_classes import Valor, Neuronio, Camada, MLP, cross_entropy

### Carregando o dataset

In [65]:
df = pd.read_csv("../Data_Frames/heart.csv")
df


,Age,ChestPainType,RestingBP,Cholesterol,RestingECG,MaxHR,HeartDisease
0,40,ATA,140,289,Normal,172,0
1,49,NAP,160,180,Normal,156,1
2,37,ATA,130,283,ST,98,0
3,48,ASY,138,214,Normal,108,1
4,54,NAP,150,195,Normal,122,0
...,...,...,...,...,...,...,...
913,45,TA,110,264,Normal,132,1
914,68,ASY,144,193,Normal,141,1
915,57,ASY,130,131,Normal,115,1
916,57,ATA,130,236,LVH,174,1


### Aplicando OneHotEncoder nas colunas:

##### ChestPainType

In [66]:
ChestPainType = df["ChestPainType"]

ChestPainType = ChestPainType.values.reshape(-1, 1)
 
encoder = OneHotEncoder(sparse_output=False)
encoder.fit(ChestPainType)
 
ChestPainType_onehot = encoder.transform(ChestPainType)

column_names = encoder.get_feature_names_out(["ChestPainType"])
df_onehot_pain = pd.DataFrame(ChestPainType_onehot, columns=column_names)

df = df.drop(columns=["ChestPainType"])

df = pd.concat([df, df_onehot_pain], axis=1)

##### RestingECG

In [67]:
RestingECG = df["RestingECG"]

RestingECG = RestingECG.values.reshape(-1, 1)
 
encoder = OneHotEncoder(sparse_output=False)
encoder.fit(RestingECG)
 
RestingECG_onehot = encoder.transform(RestingECG)

column_names = encoder.get_feature_names_out(["RestingECG"])
df_onehot_resting = pd.DataFrame(RestingECG_onehot, columns=column_names)

df = df.drop(columns=["RestingECG"])

df = pd.concat([df, df_onehot_resting], axis=1)

In [68]:
df

,Age,RestingBP,Cholesterol,MaxHR,HeartDisease,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST
0,40,140,289,172,0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,49,160,180,156,1,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,37,130,283,98,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,48,138,214,108,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,54,150,195,122,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,110,264,132,1,0.0,0.0,0.0,1.0,0.0,1.0,0.0
914,68,144,193,141,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0
915,57,130,131,115,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0
916,57,130,236,174,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [69]:
entradas = df.drop(columns=["HeartDisease"]).values
saidas = df["HeartDisease"].values.reshape(-1, 1)

### Dividindo os dados em treino, teste e validação

In [70]:
TAMANHO_TESTE_E_VALIDACAO = 0.1 

# Alocando 10% dos dados para teste
entradas_temp, entradas_teste, saidas_temp, saidas_teste = train_test_split(
    entradas, saidas, test_size=TAMANHO_TESTE_E_VALIDACAO, random_state=36)

# Alocando 10% dos dados para validação
entradas_treino, entradas_val, saidas_treino, saidas_val = train_test_split(
    entradas_temp, saidas_temp, test_size=TAMANHO_TESTE_E_VALIDACAO, random_state=36)

### Normalizando os dados

In [71]:
normalizador = MinMaxScaler()
entradas_treino = normalizador.fit_transform(entradas_treino)
entradas_val = normalizador.transform(entradas_val)
entradas_teste = normalizador.transform(entradas_teste)

### Treinando a modelo neural

In [72]:
epocas = 1000
taxa_aprendizado = 0.1
modelo = MLP(num_entradas=11, tamanhos_ocultos=[3, 2, 1])

for epoca in range(epocas):
    # Previsões no conjunto de treino
    saidas_preditas_treino = []
    for amostra in entradas_treino:
        entrada_valores = [Valor(x) for x in amostra]
        saidas_preditas_treino.append(modelo(entrada_valores))
    
    # Calcular perda
    perda_treino = cross_entropy(saidas_treino.flatten(), saidas_preditas_treino)

    # Zerar gradientes e faz backpropagation
    for p in modelo.params():
        p.grad = 0.0
    perda_treino.backward()

    # Atualizar pesos
    for p in modelo.params():
        p.data -= taxa_aprendizado * p.grad

    # Validação
    saidas_preditas_val = []
    for amostra in entradas_val:
        entrada_valores = [Valor(x) for x in amostra]
        saidas_preditas_val.append(modelo(entrada_valores))
    perda_val = cross_entropy(saidas_val.flatten(), saidas_preditas_val)

    print(f"Época {epoca:03d} — Perda Treino: {perda_treino.data:.4f} | Perda Validação: {perda_val.data:.4f}")


Época 000 — Perda Treino: 0.6904 | Perda Validação: 0.6828
Época 001 — Perda Treino: 0.6887 | Perda Validação: 0.6812
Época 002 — Perda Treino: 0.6872 | Perda Validação: 0.6798
Época 003 — Perda Treino: 0.6857 | Perda Validação: 0.6784
Época 004 — Perda Treino: 0.6843 | Perda Validação: 0.6771
Época 005 — Perda Treino: 0.6830 | Perda Validação: 0.6757
Época 006 — Perda Treino: 0.6818 | Perda Validação: 0.6744
Época 007 — Perda Treino: 0.6805 | Perda Validação: 0.6728
Época 008 — Perda Treino: 0.6791 | Perda Validação: 0.6711
Época 009 — Perda Treino: 0.6775 | Perda Validação: 0.6694
Época 010 — Perda Treino: 0.6756 | Perda Validação: 0.6676
Época 011 — Perda Treino: 0.6736 | Perda Validação: 0.6656
Época 012 — Perda Treino: 0.6714 | Perda Validação: 0.6631
Época 013 — Perda Treino: 0.6686 | Perda Validação: 0.6601
Época 014 — Perda Treino: 0.6655 | Perda Validação: 0.6573
Época 015 — Perda Treino: 0.6624 | Perda Validação: 0.6542
Época 016 — Perda Treino: 0.6588 | Perda Validação: 0.65

### Testando a modelo neural para dados de teste

In [81]:
saidas_preditas_teste = []
for amostra in entradas_teste:
    entrada_teste = [Valor(x) for x in amostra]
    saidas_preditas_teste.append(modelo(entrada_teste))

perda_teste = cross_entropy(saidas_teste.flatten(), saidas_preditas_teste)

print(f"Perda teste: {perda_teste.data:.4f}")

Perda teste: 0.6731


### Conclusão

A rede neural adaptada para resolver problemas de classificação mostrou-se muito útil, a capaciade de ter um modelo poderoso que consegue prever dados de classificação é de grande utilidade para diversas áreas, como a área da saúde com esse dataset que faz uma previsão em relação a uma doença cardíaca ou até mesmo em outras áreas da ciência.

### Referências

Cross-entropy. Disponível em: <https://en.wikipedia.org/wiki/Cross-entropy>.

CECCON, D. Funções de ativação: definição, características, e quando usar cada uma – IA Expert Academy. Disponível em: <https://iaexpert.academy/2020/05/25/funcoes-de-ativacao-definicao-caracteristicas-e-quando-usar-cada-uma/>.